# [Joining Data in SQL](https://www.datacamp.com/completed/statement-of-accomplishment/course/5d66f08968c3328e8fe73a0a725fb697a6425e42)

## Contents
  * [Inner Joins](#Inner-Joins-🔝)
  * [Outer, Cross, and Self Joins](#Outer,-Cross,-and-Self-Joins-🔝)
  * [Set Theory](#Set-Theory-🔝)
  * [Subqueries](#Subqueries-🔝)

In [1]:
%load_ext sql

%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 10
%config SqlMagic.feedback = 0

Found pyproject.toml from '/Users/adam/@/datacamp'

In [2]:
import duckdb

conn = duckdb.connect(database=':memory:', read_only=False)
%sql conn

# load data
%sql CREATE TABLE cities AS SELECT * FROM 'cities.csv';
%sql CREATE TABLE countries AS SELECT * FROM 'countries.csv';
%sql CREATE TABLE currencies AS SELECT * FROM 'currencies.csv';
%sql CREATE TABLE economies AS SELECT * FROM 'economies.csv';
%sql CREATE TABLE economies2015 AS SELECT * FROM 'economies2015.csv';
%sql CREATE TABLE economies2019 AS SELECT * FROM 'economies2019.csv';
%sql CREATE TABLE languages AS SELECT * FROM 'languages.csv';
%sql CREATE TABLE populations AS SELECT * FROM 'populations.csv';

Count
434


## Inner Joins [🔝](#Contents)

_Inner_ joins look for records in both tables which match the join condition, specified in the `ON` clause.

Like column names, tables can be aliased using `AS`. In Postgres, `AS` is optional unless the alias is a reserved word. A convention is to use `AS` with column names, but not with table names, and to not use reserved words as aliases to avoid confusion.

<img src="images/join1_key.png" alt="join1_key" width="320" />
<img src="images/inner_join2.png" alt="inner_join2" width="640" />

In [3]:
%%sql
SELECT
  cities.name AS city,
  countries.name AS country,
  countries.region
FROM cities
INNER JOIN countries ON countries.code = cities.country_code;

city,country,region
Abidjan,Cote d'Ivoire,Western Africa
Abu Dhabi,United Arab Emirates,Middle East
Abuja,Nigeria,Western Africa
Accra,Ghana,Western Africa
Addis Ababa,Ethiopia,Eastern Africa
Ahmedabad,India,Southern and Central Asia
Alexandria,Egypt,Northern Africa
Algiers,Algeria,Northern Africa
Ankara,Turkey,Middle East
Auckland,New Zealand,Australia and New Zealand


In [4]:
%%sql
SELECT
  c.code AS country_code,
  c.name,
  e.year,
  e.inflation_rate
FROM countries c
INNER JOIN economies e ON e.code = c.code;

country_code,name,year,inflation_rate
AFG,Afghanistan,2010,2.179
AFG,Afghanistan,2015,-1.549
AGO,Angola,2010,14.48
AGO,Angola,2015,10.287
ALB,Albania,2010,3.605
ALB,Albania,2015,1.896
ARE,United Arab Emirates,2010,0.878
ARE,United Arab Emirates,2015,4.07
ARG,Argentina,2010,10.461
ARG,Argentina,2015,None


### `USING`

If both tables have the same column name, we can use the `USING` clause instead of `ON` to specify the join condition.

In [5]:
%%sql
SELECT
  c.name AS country,
  l.name AS language,
  official -- unambiguous columns don't need table aliases
FROM countries c
INNER JOIN languages l USING(code);

country,language,official
Afghanistan,Dari,True
Afghanistan,Pashto,True
Afghanistan,Turkic,False
Afghanistan,Other,False
Albania,Albanian,True
Albania,Greek,False
Albania,Other,False
Albania,unspecified,False
Algeria,Arabic,True
Algeria,French,False


### Cardinality

In database design, _cardinality_ describes the relationships between tables. The three types of cardinality are:

#### One-to-one (1:1)

Each record in Table A is related to one and only one record in Table B. For example, **1** person can have only **1** social security number, and each SSN is unique.

#### One-to-many (1:N)

Each record in Table A is related to one or more records in Table B. For example, **1** customer can have **many** orders, but each order can only be associated with a single customer.

#### Many-to-many (M:N)

Each record in Table A is related to one or more records in Table B, and each record in Table B is related to one or more records in Table A. For example, a student can be enrolled in **many** classes; a class can have **many** students.

In [6]:
%%sql
SELECT
  c.name AS country,
  l.name AS language
FROM countries c
INNER JOIN languages l USING(code)
ORDER BY language;

country,language
Ethiopia,Afar
Eritrea,Afar
Djibouti,Afar
South Africa,Afrikaans
Namibia,Afrikaans
Ghana,Akyem
Macedonia,Albanian
Albania,Albanian
Switzerland,Albanian
France,Alsatian


### Multiple Joins

You can chain together multiple joins to join more than two tables.

In [7]:
%%sql
SELECT
  c.name,
  e.year,
  p.fertility_rate,
  e.unemployment_rate
FROM countries c
INNER JOIN populations p ON p.country_code = c.code
INNER JOIN economies e ON e.code = p.country_code AND e.year = p.year;

name,year,fertility_rate,unemployment_rate
Afghanistan,2010,5.746,None
Afghanistan,2015,4.653,None
Angola,2010,6.416,None
Angola,2015,5.996,None
Albania,2010,1.663,14.0
Albania,2015,1.793,17.1
United Arab Emirates,2010,1.868,None
United Arab Emirates,2015,1.767,None
Argentina,2010,2.37,7.75
Argentina,2015,2.308,None


## Outer, Cross, and Self Joins [🔝](#Contents)

### Left Joins

The first type of _outer_ join is the _left_ join. A left join returns all records from the left table, and the matched records from the right table. In the case where there is no match, the right side will contain `NULL`.

Note that `LEFT JOIN` and `LEFT OUTER JOIN` are synonymous (the `OUTER` keyword is optional).

<img src="images/left_join2.png" alt="left_join2" width="640" />

In [8]:
%%sql
SELECT
  c1.name AS city,
  c2.code,
  c2.name AS country,
  c2.region,
  c1.city_proper_pop
FROM cities c1
LEFT JOIN countries c2 ON c1.country_code = c2.code
ORDER BY code DESC;

city,code,country,region,city_proper_pop
Harare,ZWE,Zimbabwe,Eastern Africa,1606000
Lusaka,ZMB,Zambia,Eastern Africa,1742979
Ekurhuleni,ZAF,South Africa,Southern Africa,3178470
Cape Town,ZAF,South Africa,Southern Africa,3740026
Johannesburg,ZAF,South Africa,Southern Africa,4434827
Durban,ZAF,South Africa,Southern Africa,3442361
Sana'a,YEM,Yemen,Middle East,1937451
Hanoi,VNM,Vietnam,Southeast Asia,6844100
Ho Chi Minh City,VNM,Vietnam,Southeast Asia,7681700
Caracas,VEN,Venezuela,South America,1943901


In [9]:
%%sql
SELECT
  c.region,
  AVG(e.gdp_percapita) AS avg_gdp
FROM countries c
LEFT JOIN economies e USING(code)
WHERE year = 2010
GROUP BY c.region
ORDER BY avg_gdp DESC;

region,avg_gdp
Western Europe,58130.962857142855
Nordic Countries,57073.998
North America,47911.509999999995
Australia and New Zealand,44792.384999999995
British Islands,43588.33
Eastern Asia,24962.807999999997
Southern Europe,22926.410909090908
Middle East,18204.64176470588
Baltic Countries,12631.029999999999
Caribbean,11413.339461538462


### Right Joins

A _right_ join returns all records from the right table, and the matched records from the left table. In the case where there is no match, the left side will contain `NULL`.

Right joins are less common than left joins as you can always rewrite a right join as a left join to make it easier to read.

<img src="images/right_join2.png" alt="right_join2" width="640" />

In [10]:
%%sql
SELECT
  c.name AS country,
  l.name AS language,
  l.percent
FROM languages l
RIGHT JOIN countries c USING(code)
ORDER BY language;

country,language,percent
Ethiopia,Afar,1.7
Eritrea,Afar,None
Djibouti,Afar,None
South Africa,Afrikaans,13.5
Namibia,Afrikaans,10.4
Ghana,Akyem,3.2
Macedonia,Albanian,25.1
Albania,Albanian,98.8
Switzerland,Albanian,3.0
France,Alsatian,None


### Full Joins

A _full_ join combines a left join and a right join. It returns all records from both tables and matches up records where possible. In the case where there is no match, the side will contain `NULL`.

<img src="images/full_join2.png" alt="full_join2" width="640" />

In [11]:
%%sql
SELECT
  c1.name AS country,
  c1.code,
  c1.region,
  c2.basic_unit
FROM countries c1
FULL JOIN currencies c2 USING (code)
WHERE c1.region = 'North America' OR country IS NULL
ORDER BY c1.region;

country,code,region,basic_unit
Bermuda,BMU,North America,Bermudian dollar
Canada,CAN,North America,Canadian dollar
United States,USA,North America,United States dollar
Greenland,GRL,North America,None
None,None,None,Australian dollar
None,None,None,East Caribbean dollar
None,None,None,Falkland Islands pound
None,None,None,New Zealand dollar
None,None,None,Saint Helena pound
None,None,None,United States dollar


In [12]:
%%sql
SELECT
  c1.name AS country,
  c1.region,
  l.name AS language,
  c2.basic_unit,
  c2.frac_unit
FROM countries c1
FULL JOIN languages l ON c1.code = l.code
FULL JOIN currencies c2 ON c1.code = c2.code
WHERE c1.region LIKE 'M%esia';

country,region,language,basic_unit,frac_unit
Kiribati,Micronesia,English,Australian dollar,Cent
Marshall Islands,Micronesia,Other,United States dollar,Cent
Nauru,Micronesia,Other,Australian dollar,Cent
Palau,Micronesia,Other,United States dollar,Cent
Papua New Guinea,Melanesia,Other,Papua New Guinean kina,Toea
Solomon Islands,Melanesia,indigenous,Solomon Islands dollar,Cent
New Caledonia,Melanesia,Other,CFP franc,Centime
Vanuatu,Melanesia,Other,Vanuatu vatu,None
Guam,Micronesia,Other,None,None
"Micronesia, Federated States of",Micronesia,Kapingamarangi,None,None


### Cross Joins

_Cross_ joins are different from inner and outer joins in that they don't have a join condition (i.e., no `ON` or `USING`). Every row in the left table is paired with every row in the right table. This is known as a _Cartesian product_.

<img src="images/cross_join.png" alt="cross_join" width="320" />

In [13]:
%%sql
SELECT
  c.name AS country,
  l.name AS language
FROM countries c
CROSS JOIN languages l
WHERE c.code in ('PAK','IND') AND l.code in ('PAK','IND');

country,language
Pakistan,Punjabi
Pakistan,Sindhi
Pakistan,Saraiki
Pakistan,Pashto
Pakistan,Urdu
Pakistan,Balochi
Pakistan,Hindko
Pakistan,Brahui
Pakistan,English
Pakistan,Burushaski


### Self Joins

A _self_ join is a type of join where a table is joined to itself. If you wanted to see how the population of a country changed from 2010 to 2015, you could use a self join.

In [14]:
%%sql
SELECT
  p1.country_code,
  p1.size AS size2010,
  p2.size AS size2015
FROM populations p1
INNER JOIN populations p2 ON p1.country_code = p2.country_code
WHERE p1.year = 2010 AND p2.year = 2015;

country_code,size2010,size2015
ABW,101597,103889
AFG,27962207,32526562
AGO,21219954,25021974
ALB,2913021,2889167
AND,84419,70473
ARE,8329453,9156963
ARG,41222875,43416755
ARM,2963496,3017712
ASM,55636,55538
ATG,87233,91818


## Set Theory [🔝](#Contents)

SQL has 3 main set operations: `UNION`, `INTERSECT`, and `EXCEPT`.

Set operations _operate_ on two or more result sets; joins _join_ two tables together.

<img src="images/venn.png" alt="venn" width="320" />

### `UNION`

`UNION` returns all unique records from both result sets. It is important to note that `UNION` only selects unique records whereas `UNION ALL` selects all records.

<img src="images/union.png" alt="union" width="320" />
<img src="images/union_all.png" alt="union_all" width="320" />

Set operations do not have a join condition, but the result sets must have the same number of columns and compatible data types.

<img src="images/union_syntax_2.png" alt="union_syntax_2" width="480" />

In [15]:
%%sql
SELECT * FROM economies2015
UNION
SELECT * FROM economies2019
ORDER BY code, year;

code,year,income_group,gross_savings
ABW,2015,High income,14.86785151
AGO,2015,Lower middle income,25.02132654
AGO,2019,Lower middle income,25.52484772
ALB,2015,Upper middle income,16.8639814
ALB,2019,Upper middle income,14.4998255
ARG,2015,Upper middle income,14.28730262
ARG,2019,Upper middle income,14.28529512
ARM,2015,Upper middle income,18.43038559
ARM,2019,Upper middle income,9.815573922
ATG,2015,High income,7.051264504


In [16]:
%%sql
SELECT code, year FROM economies
UNION ALL
SELECT country_code, year FROM populations -- the column names come from the first query
ORDER BY code, year;

code,year
ABW,2010
ABW,2015
AFG,2010
AFG,2010
AFG,2015
AFG,2015
AGO,2010
AGO,2010
AGO,2015
AGO,2015


### `INTERSECT`

`INTERSECT` returns all distinct records that are in both result sets.

Unlike `INNER JOIN`, `INTERSECT` will not return duplicates.

<img src="images/intersect_two_columns.png" alt="intersect_two_columns" width="480" />

In [17]:
%%sql
-- all cities and countries that share the same name
SELECT name FROM cities
INTERSECT
SELECT name FROM countries;

name
Singapore


### `EXCEPT`

`EXCEPT` returns all distinct records from the first result set that are not in the second result set.

<img src="images/except.png" alt="except" width="480" />

In [18]:
%%sql
-- all cities that do not have the same name as a country
SELECT name FROM cities
EXCEPT
SELECT name FROM countries
ORDER BY name;

name
Abidjan
Abu Dhabi
Abuja
Accra
Addis Ababa
Ahmedabad
Alexandria
Algiers
Almaty
Ankara


## Subqueries [🔝](#Contents)

### Semi Join

A _semi_ join returns all records from the left table where a match exists in the right table.

<img src="images/semi2.png" alt="semi2" width="480" />

In [19]:
%%sql
SELECT DISTINCT name FROM languages
WHERE languages.code IN
  (SELECT code FROM countries
  WHERE countries.region = 'Middle East')
ORDER BY name;

name
Arabic
Aramaic
Armenian
Azerbaijani
Azeri
Baluchi
Bulgarian
Circassian
English
Farsi


### Anti Join

An _anti_ join returns all records from the left table where a match does not exist in the right table (in other words, the opposite of a semi join).

<img src="images/anti2.png" alt="anti2" width="480" />

In [20]:
%%sql
SELECT code, name FROM countries
WHERE countries.continent = 'Oceania'
AND code NOT IN (SELECT code FROM currencies);

code,name
ASM,American Samoa
FJI,Fiji Islands
GUM,Guam
FSM,"Micronesia, Federated States of"
MNP,Northern Mariana Islands


### `WHERE` Subqueries

In [21]:
%%sql
SELECT * FROM populations
WHERE year = 2015 AND life_expectancy > 1.15 * (
  SELECT AVG(life_expectancy) FROM populations
  WHERE year = 2015
);

pop_id,country_code,year,fertility_rate,life_expectancy,size
21,AUS,2015,1.833,82.4512195121951,23789752
376,CHE,2015,1.54,83.1975609756098,8281430
356,ESP,2015,1.32,83.3804878048781,46443994
134,FRA,2015,2.01,82.6707317073171,66538391
170,HKG,2015,1.195,84.2780487804878,7305700
174,ISL,2015,1.93,82.8609756097561,330815
190,ITA,2015,1.37,83.490243902439,60730582
194,JPN,2015,1.46,83.8436585365854,126958472
340,SGP,2015,1.24,82.5951219512195,5535002
374,SWE,2015,1.88,82.5512195121951,9799186


In [22]:
%%sql
SELECT name, country_code, urbanarea_pop FROM cities
WHERE name IN (SELECT capital FROM countries)
ORDER BY urbanarea_pop DESC;

name,country_code,urbanarea_pop
Beijing,CHN,21516000
Dhaka,BGD,14543124
Tokyo,JPN,13513734
Moscow,RUS,12197596
Cairo,EGY,10230350
Kinshasa,COD,10130000
Jakarta,IDN,10075310
Seoul,KOR,9995784
Mexico City,MEX,8974724
Lima,PER,8852000


### `SELECT` Subqueries

In [23]:
%%sql
SELECT
  countries.name AS country,
  (
    SELECT COUNT(*) FROM cities
    WHERE countries.code = cities.country_code
  ) AS cities_num
FROM countries
ORDER BY cities_num DESC, country;

country,cities_num
China,36
India,18
Japan,11
Brazil,10
Pakistan,9
United States,9
Indonesia,7
Russian Federation,7
South Korea,7
Iran,6


### `FROM` Subqueries

In [24]:
%%sql
SELECT
  local_name,
  sub.lang_num
FROM
  countries,
  (
    SELECT
      code,
      COUNT(*) AS lang_num
    FROM languages
    GROUP BY code
  ) AS sub
WHERE countries.code = sub.code
ORDER BY lang_num DESC;

local_name,lang_num
Zambia,19
YeItyop´iya,16
Zimbabwe,16
Bharat/India,14
Nepal,14
France,13
Mali,13
South Africa,13
Angola,12
Malawi,12
